In [31]:
b

[51,
 74,
 79,
 117,
 128,
 129,
 155,
 157,
 159,
 167,
 175,
 176,
 177,
 178,
 179,
 208,
 241,
 256,
 278]

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [35]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [36]:
# !pip install eo-learn
# !pip install geopandas
# !pip install sentinelhub

In [37]:
#!sentinelhub.config --instance_id d5f19921-fa10-4068-8ef6-0e131af6ccd9

In [38]:
from eolearn.core import EOTask, EOPatch, LinearWorkflow, Dependency, FeatureType
from eolearn.core import OverwritePermission
# We'll use Sentinel-2 imagery (Level-1C) provided through Sentinel Hub
# If you don't know what `Level 1C` means, don't worry. It doesn't matter.
from eolearn.io import S2L1CWCSInput 
from eolearn.core import LoadFromDisk, SaveToDisk

# cloud detection
from eolearn.mask import AddCloudMaskTask, get_s2_pixel_cloud_detector
from eolearn.mask import AddValidDataMaskTask

# filtering of scenes
from eolearn.features import SimpleFilterTask

# burning the vectorised polygon to raster
from eolearn.geometry import VectorToRaster

# The golden standard: numpy and matplotlib
import numpy as np

# import matplotlib TODO
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# For manipulating geo-spatial vector dataset (polygons of nominal water extent)
import geopandas as gpd

# Image manipulations
# Our water detector is going to be based on a simple threshold 
# of Normalised Difference Water Index (NDWI) grayscale image
from skimage.filters import threshold_otsu

# Loading polygon of nominal water extent
import shapely.wkt
from shapely.geometry import Polygon

# sentinelhub-py package
from sentinelhub import BBox, CRS

from skimage.filters import sobel
from skimage.morphology import disk
from skimage.morphology import erosion, dilation, opening, closing, white_tophat
import geopandas as gpd

/Users/gokul/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [39]:
input_task = S2L1CWCSInput('TRUE-COLOR-S2-L1C', resx='1m', resy='1m', maxcc=0.5, instance_id=None)
add_ndwi = S2L1CWCSInput('NDWI')

In [40]:
cloud_classifier = get_s2_pixel_cloud_detector(average_over=2, dilation_size=1, all_bands=False)

cloud_detection = AddCloudMaskTask(cloud_classifier, 'BANDS-S2CLOUDLESS', cm_size_y='160m', cm_size_x='160m', 
                                   cmask_feature='CLM', cprobs_feature='CLP', instance_id=None)

In [41]:
def calculate_valid_data_mask(eopatch):
    return np.logical_and(eopatch.mask['IS_DATA'].astype(np.bool), 
                          np.logical_not(eopatch.mask['CLM'].astype(np.bool)))

add_valid_mask = AddValidDataMaskTask(predicate=calculate_valid_data_mask)

In [42]:
def calculate_coverage(array):
    return 1.0 - np.count_nonzero(array) / np.size(array)

class AddValidDataCoverage(EOTask):
    
    def execute(self, eopatch):
        
        valid_data = eopatch.get_feature(FeatureType.MASK, 'VALID_DATA')
        time, height, width, channels = valid_data.shape
        
        coverage = np.apply_along_axis(calculate_coverage, 1, valid_data.reshape((time, height * width * channels)))
        
        eopatch.add_feature(FeatureType.SCALAR, 'COVERAGE', coverage[:, np.newaxis])
        return eopatch
    
add_coverage = AddValidDataCoverage()

In [43]:
class ValidDataCoveragePredicate:
    
    def __init__(self, threshold):
        self.threshold = threshold
        
    def __call__(self, array):
        return calculate_coverage(array) < self.threshold
    
remove_cloudy_scenes = SimpleFilterTask((FeatureType.MASK, 'VALID_DATA'), ValidDataCoveragePredicate(0.2))

In [44]:
class WaterDetector(EOTask):
    
    @staticmethod
    def detect_water(ndwi):
        """
        Very simple water detector based on Otsu thresholding method of NDWI.
        """
        otsu_thr = 1.0
        if len(np.unique(ndwi)) > 1:
            otsu_thr = threshold_otsu(ndwi)

        return ndwi > otsu_thr

    def execute(self, eopatch):
        water_masks = np.asarray([self.detect_water(ndwi[...,0]) for ndwi in eopatch.data['NDWI']])
        
        # we're only interested in the water within the dam borders
        water_masks = water_masks[...,np.newaxis] * eopatch.mask_timeless['NOMINAL_WATER']
        
        water_levels = np.asarray([np.count_nonzero(mask)/np.count_nonzero(eopatch.mask_timeless['NOMINAL_WATER']) 
                                   for mask in water_masks])
        
        eopatch.add_feature(FeatureType.MASK, 'WATER_MASK', water_masks)
        eopatch.add_feature(FeatureType.SCALAR, 'WATER_LEVEL', water_levels[...,np.newaxis])
        
        return eopatch
    
water_detection = WaterDetector()

In [45]:
time_interval = ['2010-01-01','2019-08-31']

In [46]:
poly = gpd.read_file("source.geojson")

In [47]:
def plot_rgb_w_water(eopatch, idx):
    ratio = np.abs(eopatch.bbox.max_x - eopatch.bbox.min_x) / np.abs(eopatch.bbox.max_y - eopatch.bbox.min_y)
    fig, ax = plt.subplots(figsize=(ratio * 10, 10))
    
    ax.imshow(eopatch.data['TRUE-COLOR-S2-L1C'][idx])
    
    observed = closing(eopatch.mask['WATER_MASK'][idx,...,0], disk(1))
    nominal = sobel(eopatch.mask_timeless['NOMINAL_WATER'][...,0])
    observed = sobel(observed)
    nominal = np.ma.masked_where(nominal == False, nominal)
    observed = np.ma.masked_where(observed == False, observed)
    
    ax.imshow(nominal, cmap=plt.cm.Reds)
    ax.imshow(observed, cmap=plt.cm.Blues)
    ax.axis('off')

In [48]:
def plot_water_levels(eopatch, max_coverage=1.0):
    fig, ax = plt.subplots(figsize=(20, 7))

    dates = np.asarray(eopatch.timestamp)
    ax.plot(dates[eopatch.scalar['COVERAGE'][...,0] < max_coverage],
            eopatch.scalar['WATER_LEVEL'][eopatch.scalar['COVERAGE'][...,0] < max_coverage],
            'bo-', alpha=0.7)
    ax.plot(dates[eopatch.scalar['COVERAGE'][...,0] < max_coverage],
            eopatch.scalar['COVERAGE'][eopatch.scalar['COVERAGE'][...,0] < max_coverage],
            '--', color='gray', alpha=0.7)
    ax.set_ylim(0.0, 1.1)
    ax.set_xlabel('Date')
    ax.set_ylabel('Water level')
    ax.set_title('Theewaterskloof Dam Water Levels')
    ax.grid(axis='y')
    return ax

In [49]:
def call_fun(dam_nominal,dam_bbox,count):
  dam_gdf = gpd.GeoDataFrame(crs={'init':'epsg:4326'}, geometry=[dam_nominal])
  add_nominal_water = VectorToRaster(dam_gdf, (FeatureType.MASK_TIMELESS, 'NOMINAL_WATER'), values=1, 
                                   raster_shape=(FeatureType.MASK, 'IS_DATA'), raster_dtype=np.uint8)
  workflow = LinearWorkflow(input_task, add_ndwi, cloud_detection, add_nominal_water, add_valid_mask, add_coverage,
                          remove_cloudy_scenes, water_detection)
  result = workflow.execute({
      input_task: {
          'bbox': dam_bbox,
          'time_interval': time_interval
      },
  })
  patch = list(result.values())[-1]
  patch.save('patches/patches_'+str(count), overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)
  return patch

In [ ]:
for i in range(175,200):
    try:
        dam_nominal = poly.geometry[i]
        inflate_bbox = 0.1
        minx, miny, maxx, maxy = dam_nominal.bounds
        print(i)
        delx = maxx - minx
        dely = maxy - miny
        minx = minx - delx * inflate_bbox
        maxx = maxx + delx * inflate_bbox
        miny = miny - dely * inflate_bbox
        maxy = maxy + dely * inflate_bbox

        dam_bbox = BBox([minx, miny, maxx, maxy], crs=CRS.WGS84)

        dam_bbox.geometry - dam_nominal
        call_fun(dam_nominal,dam_bbox,i)
    except:
        print("error:" + str(i))

175
error:175
176
error:176
177
error:177
178


In [55]:
for i in poly.S_No_:
    print(i)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
121.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
168.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
169.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
184.0
185.

In [51]:
poly.head

<bound method NDFrame.head of                                   Name  \
0    Thamaraikulam Kathivakkam Village   
1                       Railway Pond 2   
2                       Railway Pond 1   
3                       Railway Pond 3   
4                      Sathankadu lake   
..                                 ...   
285          OMR Majestic Layout Kulam   
286    Purah Kulam - Pondicheri Pettai   
287           Rajiv Gandhi Nagar Kulam   
288       Sarayakulam Puducheri Pettai   
289      Veeravanjinathan Street Kulam   

                                           description  S_No_ Zone Ward  \
0    S.No.: 1<br>Zone: 1<br>Ward: 1<br>Latitude: 13...    1.0    1    1   
1    S.No.: 2<br>Zone: 1<br>Ward: 3<br>Latitude: 13...    2.0    1    3   
2    S.No.: 3<br>Zone: 1<br>Ward: 4<br>Latitude: 13...    3.0    1    4   
3    S.No.: 4<br>Zone: 1<br>Ward: 6<br>Latitude: 13...    4.0    1    6   
4    S.No.: 5<br>Zone: 1<br>Ward: 7<br>Latitude: 13...    5.0    1    7   
..             